# Analyze RCI correlations between cell lines

In [1]:
from datetime import datetime
print(datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)

2023-01-08 12:35:10.337949
Python 3.10.0
sklearn 1.1.2


In [2]:
try:
    from google.colab import drive
    IN_COLAB = True
    print('Running on CoLab')
    PATH='/content/drive/'
    drive.mount(PATH)
    DATA_DIR=PATH+'My Drive/data/Localization/TrainTest/'  # must end in "/"
except:
    IN_COLAB = False
    DATA_DIR = 'D:/Adjeroh/Localization/TrainTest/'   # Windows
    DATA_DIR = '/Users/jasonmiller/WVU/Localization/TrainTest/'    # Mac
print(DATA_DIR)

/Users/jasonmiller/WVU/Localization/TrainTest/


In [3]:
PC_RCI_FILE =      'CNRCI_coding_train_RCI.gc42.csv'
NC_RCI_FILE =      'CNRCI_noncoding_train_RCI.gc42.csv'

In [8]:
def load_RCI_data(filepath):
    cell_line_names = []
    cell_line_maps  = []
    with open (filepath,'r') as handle:
        header = None
        for row in handle:
            line = row.strip()
            fields = line.split(',')
            gene_id = fields[0]
            if header is None:
                header = row 
                for i in range(len(fields)): 
                    cell_line_name = fields[i]
                    cell_line_names.append(cell_line_name)
                    cell_line_genes = {}
                    cell_line_maps.append(cell_line_genes)
            else:
                for i in range(len(fields)): 
                    if i>0:  # column 0 is gene_id
                        rci_val = fields[i]
                        if rci_val != "nan":
                            real_value = float(rci_val)
                            cell_line_genes = cell_line_maps[i]
                            cell_line_genes[gene_id] = real_value 
    return cell_line_names, cell_line_maps

In [16]:
def get_union(maps):
    union = set()
    for i in range(len(maps)):
        one_map = maps[i]
        genes = one_map.keys()
        union = union.union(genes)
    return union
def get_intersection(maps):
    section = set()
    for i in range(len(maps)):
        one_map = maps[i]
        genes = set(one_map.keys())
        if i==0:
            section = genes
        else:
            section = section.intersection(genes)
    return section
def show_rci_counts_table(names,maps):
    for i in range(len(names)):
        print('%10s %5d' % (names[i], len(maps[i].keys())))
    union = get_union(cl_maps)
    print('The union of RCI values contains',len(union),'genes.')
    section = get_intersection(cl_maps)
    print('The intersection of RCI values contains',len(section),'genes.')

In [17]:
filepath = DATA_DIR + PC_RCI_FILE
cl_names,cl_maps = load_RCI_data(filepath)
print('Coding Genes: number of RCI values per cell line')
show_rci_counts_table(cl_names,cl_maps)

Coding Genes: number of RCI values per cell line
   gene_id     0
      A549 10338
   H1.hESC 13000
   HeLa.S3  6983
     HepG2  9424
    HT1080  9575
     HUVEC 10257
     MCF.7 11060
  NCI.H460  8441
      NHEK  8738
  SK.MEL.5  8407
   SK.N.DZ  8510
   SK.N.SH  9689
   GM12878  9931
      K562  6676
    IMR.90  4001
The union of RCI values contains 13978 genes.
The intersection of RCI values contains 0 genes.


In [18]:
filepath = DATA_DIR + NC_RCI_FILE
cl_names,cl_maps = load_RCI_data(filepath)
print('Noncoding Genes: number of RCI values per cell line')
show_rci_counts_table(cl_names,cl_maps)

Noncoding Genes: number of RCI values per cell line
   gene_id     0
      A549  1447
   H1.hESC  3327
   HeLa.S3   919
     HepG2  1362
    HT1080   936
     HUVEC  1488
     MCF.7  2156
  NCI.H460   610
      NHEK  1080
  SK.MEL.5   534
   SK.N.DZ   603
   SK.N.SH  1647
   GM12878  1698
      K562   958
    IMR.90   390
The union of RCI values contains 4662 genes.
The intersection of RCI values contains 0 genes.
